In [1]:
# 为什么调用tensorboard 后面加--host=127.0.0.1才能打开网址。
# 以前也不会这样啊 难道是我用爬虫时更改了ip？
import tensorflow as tf
import os
# tarfile模块的主要作用是用来加压缩和解压缩文件
# 其压缩文件的类型可以是.tar  |  .tar.gz | .tgz | .tar.bz2 等
import tarfile
import requests

In [2]:
#inception模型下载地址
inception_pretrain_model_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

#模型存放地址
inception_pretrain_model_dir = "inception_model"
if not os.path.exists(inception_pretrain_model_dir):
    os.makedirs(inception_pretrain_model_dir)
    
#获取文件名，以及文件路径
filename = inception_pretrain_model_url.split('/')[-1]
#os.path.join 组合括号内的路径 即inception_pretrain_model_dir/filename
filepath = os.path.join(inception_pretrain_model_dir, filename)

#下载模型
#判断模型是否存在并下载
if not os.path.exists(filepath):
    print("download: ", filename)
    #stream 流，用于下载文件
    r = requests.get(inception_pretrain_model_url, stream=True)
    with open(filepath, 'wb') as f:
        #分段写入文件
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
print("finish: ", filename)

#解压文件
tarfile.open(filepath, 'r:gz').extractall(inception_pretrain_model_dir)
 
#模型结构存放文件
log_dir = 'inception_log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

#classify_image_graph_def.pb为google训练好的模型
inception_graph_def_file = os.path.join(inception_pretrain_model_dir, 'classify_image_graph_def.pb')
with tf.Session() as sess:
    #创建一个图来存放google训练好的模型
    with tf.gfile.FastGFile(inception_graph_def_file, 'rb') as f:
        #这一部分不懂具体操作
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
    #保存图的结构
    writer = tf.summary.FileWriter(log_dir, sess.graph)
    writer.close()

download:  inception-2015-12-05.tgz
finish:  inception-2015-12-05.tgz
Instructions for updating:
Use tf.gfile.GFile.
